In [2]:
import pandas as pd
from miners.congresspeople import CongressAPI
import asyncio
import logging
import os

In [3]:
data = pd.read_json('data/raw/congresspeople_2023-11-19 22:44:23.419408.json')

proposicoes_files = os.listdir('data/raw/proposicoes')
proposicoes = pd.DataFrame()
for file in proposicoes_files:
    proposicao = pd.json_normalize(pd.read_json(f'data/raw/proposicoes/{file}')['dados'])
    proposicoes = pd.concat([proposicoes, proposicao])
proposicoes

,id,uri,siglaTipo,numero,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords,...,ultimoStatus.siglaOrgao,ultimoStatus.uriOrgao,ultimoStatus.regime,ultimoStatus.descricaoTramitacao,ultimoStatus.idTipoTramitacao,ultimoStatus.descricaoSituacao,ultimoStatus.idSituacao,ultimoStatus.despacho,ultimoStatus.apreciacao,ultimoStatus.url
0,112546,https://dadosabertos.camara.leg.br/api/v2/prop...,SBT,1,0,255,Substitutivo,,,,...,,,,Apresentação de Proposição,100,,,Apresentação do Substitutivo pelo Dep. Saraiva...,Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...
1,339738,https://dadosabertos.camara.leg.br/api/v2/prop...,RDF,2,0,197,Redação Final,,,,...,,,,Apresentação de Proposição,100,,,"Apresentação da Redação Final, RDF 2 CCJC, pel...",Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...
2,339741,https://dadosabertos.camara.leg.br/api/v2/prop...,TVR,1275,2007,492,Autorização - Rádio Comunitária - Dez anos,Submete à apreciação do Congresso Nacional o a...,,"Rádio comunitária, Açailândia, (MA).",...,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,Art. 223 - CF,Notificações,1005,Transformado em nova proposição,1230,Transformado no PDC 105/2007.,Proposição Sujeita à Apreciação Conclusiva (Pa...,
3,339742,https://dadosabertos.camara.leg.br/api/v2/prop...,TVR,1276,2007,492,Autorização - Rádio Comunitária - Dez anos,Submete à apreciação do Congresso Nacional o a...,,"Rádio comunitária, Lins, (SP).",...,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,Art. 223 - CF,Notificações,1005,Transformado em nova proposição,1230,Transformado no PDC 106/2007.,Proposição Sujeita à Apreciação Conclusiva (Pa...,
4,339743,https://dadosabertos.camara.leg.br/api/v2/prop...,TVR,1277,2007,492,Autorização - Rádio Comunitária - Dez anos,Submete à apreciação do Congresso Nacional o a...,,"Rádio comunitária, Acaraú, (CE).",...,MESA,https://dadosabertos.camara.leg.br/api/v2/orga...,Art. 223 - CF,Notificações,1005,Transformado em nova proposição,1230,Transformado no PDC 577/2008.,Proposição Sujeita à Apreciação Conclusiva (Pa...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30310,2212339,https://dadosabertos.camara.leg.br/api/v2/prop...,SBT-A,1,0,701,Substitutivo adotado pela Comissão,Acrescenta alíneas ao inciso III do art. 1º da...,,,...,,,,Apresentação de Proposição,100,,,Apresentação do Substitutivo adotado pela Comi...,Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...
30311,2214445,https://dadosabertos.camara.leg.br/api/v2/prop...,PRL,1,0,190,Parecer do Relator,"Parecer do Relator, Dep. Sandes Júnior, pela r...",,,...,,,,Análise Parecer,319,,,"Parecer do Relator, Dep. Sandes Júnior, pela r...",Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...
30312,2214447,https://dadosabertos.camara.leg.br/api/v2/prop...,PRL,1,0,190,Parecer do Relator,"Parecer do Relator, Dep. Hugo Leal (PSD-RJ), p...",,,...,,,,Análise Parecer,319,,,"Parecer do Relator, Dep. Hugo Leal (PSD-RJ), p...",Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...
30313,2214448,https://dadosabertos.camara.leg.br/api/v2/prop...,PAR,1,0,187,Parecer de Comissão,,,,...,,,,Apresentação de Proposição,100,,,"Apresentação do Parecer de Comissão n. 1 CVT, ...",Indefinida,https://www.camara.leg.br/proposicoesWeb/prop_...


In [28]:
for descricao in ['PL', 'PEC', 'PLP', 'MPV', 'PDC', 'PRC', 'REQ', 'RQN', 'RVD']:
    print(descricao, proposicoes[proposicoes['siglaTipo'] == descricao]['descricaoTipo'].unique(), proposicoes[proposicoes['siglaTipo'] == descricao]['descricaoTipo'].count())

proposicoes = proposicoes[proposicoes['siglaTipo'].isin(['PL', 'PEC', 'PLP', 'MPV', 'PDC', 'PRC', 'REQ', 'RQN', 'RVD'])]

PL ['Projeto de Lei'] 58259
PEC ['Proposta de Emenda à Constituição'] 2309
PLP ['Projeto de Lei Complementar'] 2993
MPV ['Medida Provisória'] 1175
PDC ['Projeto de Decreto Legislativo de Acordos, tratados ou atos internacionais'
 'Projeto de Decreto Legislativo'
 'Projeto de Decreto Legislativo de Concessão, Renovação e Permissão de Radio/TV'
 'Projeto de Decreto Legislativo de Programação Monetária'
 'Projeto de Decreto Legislativo de Referendo ou Plebiscito'
 'Projeto de Decreto Legislativo de Aprovação de Contas dos Presidentes'
 'Projeto de Decreto Legislativo de Sustação de Atos Normativos do Poder Executivo'
 'Projeto de Decreto Legislativo de Perempção da Concessão'
 'Projeto de Decreto Legislativo de Medida Provisória'
 'Projeto de Decreto Legislativo de Autorização do Congresso Nacional'
 'Projeto de Decreto Legislativo de Ministro do TCU'
 'Projeto de Decreto Legislativo de Alteração de Decreto Legislativo'
 'Projeto de Decreto Legislativo de Perempção'] 10242
PRC ['Projeto d

In [29]:
ids = proposicoes['id'].values
ids.sort()
ids = list(set(ids))
len(ids)

184596

In [30]:
tasks = []
api = CongressAPI(log_level=logging.DEBUG, concurrency_limit=25, sleep_spacing=0.5)
headers = {'Accept': 'application/json'}
params = {}

for id in ids:
    tasks.append(asyncio.create_task(api.make_request(f'https://dadosabertos.camara.leg.br/api/v2/proposicoes/{id}/autores', headers, params)))

results = await asyncio.gather(*tasks)

CancelledError: 